In [1]:
import scrapy 
class GITHUB(scrapy.Spider):
    name ='Github_Crawler'
    def start_requests(self):
        url ='https://github.com/topics'
        yield scrapy.Request(url,callback=self.parse)
    def parse(self,response):
        links=[]
        for a in response.css('a'):
            link = a.attrib.get('href')
            if link and "/topics/" in link:
                links.append(link)   
                
        #yield {"Links to all the topics ":links}   
        links_concat=[]
        for a in links:
            link = "https://github.com"+a
            links_concat.append(link)
        for url in links_concat:
            yield response.follow(url,self.parse1)

    def parse1(self, response):
        Articles = []
     
        article = response.css('div.markdown-body.f5.mb-2 p::text').getall()
        name=response.css('h1.h1::text').getall()
        name = ''.join(name).strip()
        
        
        link='https://github.com'+name

        yield {"Topic":name,"Description":article,"url":link}



In [2]:
from scrapy.crawler import CrawlerProcess
process = CrawlerProcess(
    settings={
    'FEED_FORMAT': 'csv',  # Output format is CSV
    'FEED_URI': 'New_Scrapped_Data.csv'}  # Save the output to a CSV file
    
)
process.crawl(GITHUB)
process.start()

2024-04-14 16:30:06 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: scrapybot)
2024-04-14 16:30:06 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.10.0, Python 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 23.2.0 (OpenSSL 3.0.13 30 Jan 2024), cryptography 41.0.3, Platform Windows-10-10.0.22631-SP0
2024-04-14 16:30:06 [scrapy.crawler] INFO: Overridden settings:
{}
2024-04-14 16:30:06 [py.warnings] WARNING: C:\Users\asus\anaconda3\Lib\site-packages\scrapy\utils\request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See th

In [2]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import scrapy
topics_url = 'https://github.com/topics'
response = requests.get(topics_url)
os.getcwd()

'C:\\Users\\asus\\Desktop\\jony\\jony'

In [3]:
page_contents = response.text

In [4]:
from bs4 import BeautifulSoup
doc = BeautifulSoup(page_contents, 'html.parser')
selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
topic_title_tags = doc.find_all('p', {'class': selection_class})

len(topic_title_tags)

30

In [5]:
desc_selector = 'f5 color-fg-muted mb-0 mt-1'
topic_desc_tags = doc.find_all('p', {'class': desc_selector})

In [6]:
topic_title_tag0 = topic_title_tags[0]
div_tag = topic_title_tag0.parent
topic_link_tags = doc.find_all('a', {'class': 'no-underline flex-1 d-flex flex-column'})
len(topic_link_tags)
topic_link_tags[0]

<a class="no-underline flex-1 d-flex flex-column" href="/topics/3d">
<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>
<p class="f5 color-fg-muted mb-0 mt-1">
          3D refers to the use of three-dimensional graphics, modeling, and animation in various industries.
        </p>
</a>

In [7]:
topic0_url = "https://github.com" + topic_link_tags[0]['href']
print(topic0_url)

https://github.com/topics/3d


In [8]:
topic_titles = []

for tag in topic_title_tags:
    topic_titles.append(tag.text)
    
print(topic_titles)

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android', 'Angular', 'Ansible', 'API', 'Arduino', 'ASP.NET', 'Atom', 'Awesome Lists', 'Amazon Web Services', 'Azure', 'Babel', 'Bash', 'Bitcoin', 'Bootstrap', 'Bot', 'C', 'Chrome', 'Chrome extension', 'Command line interface', 'Clojure', 'Code quality', 'Code review', 'Compiler', 'Continuous integration', 'COVID-19', 'C++']


In [9]:
topic_descs = []

for tag in topic_desc_tags:
    topic_descs.append(tag.text.strip())


In [10]:
topic_urls = []
base_url = 'https://github.com'

for tag in topic_link_tags:
    topic_urls.append(base_url + tag['href'])


In [11]:
import pandas as pd
topics_dict = {
    'title': topic_titles,
    'description': topic_descs,
    'url': topic_urls
}

In [12]:
topic_page_url = topic_urls[0]
topic_page_url
'https://github.com/topics/3d'
response = requests.get(topic_page_url)
response.status_code

200

In [13]:
topic_doc = BeautifulSoup(response.text, 'html.parser')

In [14]:
from bs4 import BeautifulSoup

h3_selection_class = 'f3 color-fg-muted text-normal lh-condensed'
repo_tags = topic_doc.find_all('h3', {'class': h3_selection_class})

owner = repo_tags[0].find('a').text.strip()
repo_name = repo_tags[0].find_all('a')[1].text.strip()

In [62]:
a_tags = repo_tags[0].find_all('a')

In [63]:
a_tags[0].text.strip()

'mrdoob'

In [64]:
base_url = 'https://github.com'
repo_url = base_url + a_tags[1]['href']
repo_url

'https://github.com/mrdoob/three.js'

In [65]:
star_tags = topic_doc.find_all('span', { 'class': 'Counter js-social-count'})

In [66]:
star_tags[2].text.strip()

'22.6k'

In [67]:
def parse_star_count(stars_str):
    stars_str = stars_str.strip()
    if stars_str[-1] == 'k':
        return int(float(stars_str[:-1]) * 1000)
    return int(stars_str)

In [68]:
def get_repo_info(h1_tag, star_tag):
    # returns all the required info about a repository
    a_tags = h1_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url =  base_url + a_tags[1]['href']
    stars = parse_star_count(star_tag.text.strip())
    return username, repo_name, stars, repo_url

In [69]:
get_repo_info(repo_tags[0], star_tags[2])

('mrdoob', 'three.js', 22600, 'https://github.com/mrdoob/three.js')

In [71]:
topic_repos_dict = {
    'username': [],
    'repo_name': [],
    'stars': [],
    'repo_url': []
}


for i in range(len(repo_tags)):
    repo_info = get_repo_info(repo_tags[i], star_tags[i])
    topic_repos_dict['username'].append(repo_info[0])
    topic_repos_dict['repo_name'].append(repo_info[1])
    topic_repos_dict['stars'].append(repo_info[2])
    topic_repos_dict['repo_url'].append(repo_info[3])

In [76]:
import os

def get_topic_page(topic_url):
    # Download the page
    response = requests.get(topic_url)
    # Check successful response
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    # Parse using Beautiful soup
    topic_doc = BeautifulSoup(response.text, 'html.parser')
    return topic_doc

def get_repo_info(h1_tag, star_tag):
    # returns all the required info about a repository
    a_tags = h1_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url =  base_url + a_tags[1]['href']
    stars = parse_star_count(star_tag.text.strip())
    return username, repo_name, stars, repo_url

def get_topic_repos(topic_doc):
    # Get the h1 tags containing repo title, repo URL and username
    h1_selection_class = 'f3 color-fg-muted text-normal lh-condensed'
    repo_tags = topic_doc.find_all('h3', {'class': h3_selection_class})
    # Get star tags
    star_tags = topic_doc.find_all('span', { 'class': 'Counter js-social-count'})
    
    
    
    topic_repos_dict = { 'username': [], 'repo_name': [], 'stars': [],'repo_url': []}

    # Get repo info
    for i in range(len(repo_tags)):
        repo_info = get_repo_info(repo_tags[i], star_tags[i])
        topic_repos_dict['username'].append(repo_info[0])
        topic_repos_dict['repo_name'].append(repo_info[1])
        topic_repos_dict['stars'].append(repo_info[2])
        topic_repos_dict['repo_url'].append(repo_info[3])
        
    return pd.DataFrame(topic_repos_dict)

def scrape_topic(topic_url, path):
    if os.path.exists(path):
        print("The file {} already exists. Skipping...".format(path))
        return
    topic_df = get_topic_repos(get_topic_page(topic_url))
    
    topic_df.to_csv(path, index=None)

In [77]:
def get_topic_titles(doc):
    selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
    topic_title_tags = doc.find_all('p', {'class': selection_class})
    topic_titles = []
    for tag in topic_title_tags:
        topic_titles.append(tag.text)
    return topic_titles

def get_topic_descs(doc):
    desc_selector = 'f5 color-fg-muted mb-0 mt-1'
    topic_desc_tags = doc.find_all('p', {'class': desc_selector})
    topic_descs = []
    for tag in topic_desc_tags:
        topic_descs.append(tag.text.strip())
    return topic_descs

def get_topic_urls(doc):
    topic_link_tags = doc.find_all('a', {'class': 'no-underline flex-1 d-flex flex-column'})
    topic_urls = []
    base_url = 'https://github.com'
    for tag in topic_link_tags:
        topic_urls.append(base_url + tag['href'])
    return topic_urls
    

def scrape_topics():
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    topics_dict = {
        'title': get_topic_titles(doc),
        'description': get_topic_descs(doc),
        'url': get_topic_urls(doc)
    }
    return pd.DataFrame(topics_dict)

In [78]:
def scrape_topics_repos():
    print('Scraping list of topics')
    topics_df = scrape_topics()
    
    os.makedirs('data', exist_ok=True)
    for index, row in topics_df.iterrows():
        print('Scraping top repositories for "{}"'.format(row['title']))
        scrape_topic(row['url'], 'data/{}.csv'.format(row['title']))

In [79]:
scrape_topics_repos()

2024-04-14 15:05:01 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): github.com:443


Scraping list of topics


2024-04-14 15:05:02 [urllib3.connectionpool] DEBUG: https://github.com:443 "GET /topics HTTP/1.1" 200 None
2024-04-14 15:05:02 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): github.com:443


Scraping top repositories for "3D"


2024-04-14 15:05:03 [urllib3.connectionpool] DEBUG: https://github.com:443 "GET /topics/3d HTTP/1.1" 200 None
2024-04-14 15:05:03 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): github.com:443


...

2024-04-14 15:05:50 [urllib3.connectionpool] DEBUG: https://github.com:443 "GET /topics/continuous-integration HTTP/1.1" 200 None
2024-04-14 15:05:51 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): github.com:443


Scraping top repositories for "COVID-19"


2024-04-14 15:05:52 [urllib3.connectionpool] DEBUG: https://github.com:443 "GET /topics/covid-19 HTTP/1.1" 200 None
2024-04-14 15:05:52 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): github.com:443


Scraping top repositories for "C++"


2024-04-14 15:05:54 [urllib3.connectionpool] DEBUG: https://github.com:443 "GET /topics/cpp HTTP/1.1" 200 None
